<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%86%8C%ED%98%84_%EC%84%9C%EB%B9%84%EC%8A%A42_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%95%EC%A0%9C6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 요기요 데이터 불러오기

In [6]:
yogiyo = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905.csv")
yogiyo

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,저도 모르게 은근 여기 자주 시켜먹고 있었네요ㅎ \n늘 잘먹고 있습니다,5,5,5,5.0,저희 메뉴 맛있죠?ㅎㅎ 이렇게 빠져 드시는 거에요>.<\n\n그냥 일로서 만드는 게...,제육옛날도시락＋김찌 정식＋（서비스）콜라
1,1988응답하라추억의옛날도시락-방이점,김치찌개 된장찌개 잡채\n다 잘먹었습니다,5,5,5,5.0,전부 잘 드셔 주셔서 감사합니다 ><\n\n좋은 시간을 드리고 싶었어요ㅠㅠ 열심히 ...,"돼지고기김치찌개＋대접밥,우삼겹된장찌개＋대접밥,오색빛깔 잡채"
2,1988응답하라추억의옛날도시락-방이점,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요. 다음에도 또시킬듯요,5,5,5,5.0,안녕하세요~~! :D\n\n저희 매장을 처음 찾아주신 고객님이시네요😆 처음 방문이라...,"[불맛]제육 옛날도시락,우삼겹옛날도시락＋미니냉면＋（서비스）콜라"
3,1988응답하라추억의옛날도시락-방이점,맛있습니다! 근데 국물이 안와서 아쉽습니다..,4,4,4,3.0,안녕하세요 1988응답하라추억의옛날도시락입니다.\n\n저희 측 실수로 인하여 누락이...,양념갈비 옛날도시락
4,1988응답하라추억의옛날도시락-방이점,맛있게잘먹었습니다,5,5,5,5.0,"헤헤, gh**님! 감사해요! 😊 \n\n맛있게 드셨다니 앞으로도 더 열심히 일할게...","삼겹정찬도시락,돼지고기김치찌개＋대접밥"
...,...,...,...,...,...,...,...,...
24993,놀부 부대찌개-광주우산점,먹을만하다고느낍니다,4,3,3,3.0,놀부를 이용해 주셔서 감사합니다 ^^\n큰 만족을 드리지 못한것 같아 죄송합니다.\...,NaN
24994,놀부 부대찌개-광주우산점,조리를해야하는 불편함이 있긴했지만..매장에서 먹는맛 그대로이고 양도 엄청많네요 4인...,5,5,5,5.0,저희 놀부를 이용해 주셔서감사드립니다 ^^\n집에서 조리시에도 매장과 최대한 비슷하...,NaN
24995,놀부 부대찌개-광주우산점,오랜만에 놀부부대찌게~~~\n맛있었는더ㆍ\n햄 양을 늘리심 좋을듯,5,5,4,5.0,놀부를 이용해 주셔서 감사합니다 ^^\n맛있게 드셨다니 다행입니다.\n저희는 정해진...,NaN
24996,놀부 부대찌개-광주우산점,예전에 참전한 6.25 에서 먹은 부대찌개 맛이 나네요 맛있읍니다.,5,5,5,5.0,저희 놀부부대찌개를 맛있게 드셔주셔서 감사합니다 ^^\n어디서 드시더라도 항상 맛있...,NaN


- 별점 비율 확인

In [7]:
for i in range(1, 6):
    num_rating = yogiyo[yogiyo['별점'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / yogiyo.shape[0] * 100}%")

별점 1점 개수: 111, 0.4440355228418274%
별점 2점 개수: 87, 0.34802784222737815%
별점 3점 개수: 227, 0.9080726458116649%
별점 4점 개수: 771, 3.084246739739179%
별점 5점 개수: 23802, 95.21561724937995%


## 데이터셋 분리(20% 테스트 데이터)

In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(yogiyo, test_size=0.2, random_state=SEED, stratify=yogiyo['별점'])

In [9]:
df_train.shape, df_test.shape

((19998, 8), (5000, 8))

In [10]:
for i in range(1, 6):
    num_rating = df_train[df_train['별점'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / df_train.shape[0] * 100}%")

별점 1점 개수: 89, 0.445044504450445%
별점 2점 개수: 70, 0.35003500350035005%
별점 3점 개수: 181, 0.9050905090509052%
별점 4점 개수: 617, 3.0853085308530854%
별점 5점 개수: 19041, 95.21452145214522%


In [11]:
df_train = df_train.reset_index(drop=True)
df_train

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,마라하오-죽전점,비 오는데도 배달 빨리 해주시고 감사합니다!!\n맛있어요옹,5,5,5,5.0,"안녕하세요 마라하오죽전점 입니다.\n즐거운 시간 보내시고, \n소중한 리뷰 작성해주...",[내맘대로쏙] 신메뉴 셀프 마라탕
1,놀부 부대찌개-광주우산점,2인세트인데 양도 엄청많아서 좋았어요!\n맛있게 잘 먹었습니당 ㅎㅎ,4,4,4,4.0,놀부부대찌개를 이용해주셔서 감사합니다 ^^\n맛있는 음식을 넉넉하게 드실수 있도록\...,NaN
2,킹프레소빅와플,너무맛있어요!! 자주시켜먹게되네요ㅎㅎ,5,5,5,5.0,은근히 당기는 맛들로 준비하고 있습니다\n직원들의 입맛을 통과한 맛있는 디져트와 음...,"타코야끼 7알 （일반맛）,콩빵,（추천）NEW 로제 떡볶이 2인분"
3,우리집반찬도시락-혜화막내딸지점,항상 맛있어요 서비스도쵝오,5,5,5,5.0,주문 감사드립니다 ^^ 맛있게 드시고요 환절기에 감기조심하시고요\n앞으로도 만족을 ...,"파김치,우엉,깻잎,곰피 （ 초고추장 포함 ）,봄동나물,돼지갈비찜（데워 드셔야해요）2~3人"
4,우리집반찬도시락-혜화막내딸지점,모든 반찬이 다 맛있습니다!!!,5,5,5,0.0,주문 감사드립니다 ^^ 이번 주 월요일도 화이팅입니다~\n미세먼지 조심하시고 감기도...,"연근,두부조림,단호박 샐러드"
...,...,...,...,...,...,...,...,...
19993,스시리,맛있게 먹었습니다.,5,5,3,2.0,맛있게 드셨다니 감사해요. 언제드셔도 기분좋고 맛있게 드실수있도록 준비해드릴께요. ...,"（가성비）알찬초밥11p＋미니우동,모듬롤"
19994,팔도한마당순대,맛있습니다. 양도많아요,5,5,5,5.0,정말 감사합니다,"돼지수육국밥,한판순대"
19995,돈까스회관,김포 맛집!! 항상 친절하신 사장님!!,5,5,5,0.0,ek**님\n항상 맛난 떡볶이주문 감사드립니다\n김포 맛집!!! 항상 친절하신 사...,회관 국물 떡볶이 （비조리）
19996,혼밥대왕-서대전점,짱입니다 진짜 단골이에요,5,5,5,5.0,단골 쭉~~ 해주실거죠?? ㅎㅎ 감사드립니다 ^^\n\n저희 혼밥대왕을 이용해주셔서...,간장불백비빔밥 ＋ 미니장국


In [12]:
df_train = df_train[['고객리뷰','사장댓글','별점']]
df_train.head()

,고객리뷰,사장댓글,별점
0,비 오는데도 배달 빨리 해주시고 감사합니다!!\n맛있어요옹,"안녕하세요 마라하오죽전점 입니다.\n즐거운 시간 보내시고, \n소중한 리뷰 작성해주...",5
1,2인세트인데 양도 엄청많아서 좋았어요!\n맛있게 잘 먹었습니당 ㅎㅎ,놀부부대찌개를 이용해주셔서 감사합니다 ^^\n맛있는 음식을 넉넉하게 드실수 있도록\...,4
2,너무맛있어요!! 자주시켜먹게되네요ㅎㅎ,은근히 당기는 맛들로 준비하고 있습니다\n직원들의 입맛을 통과한 맛있는 디져트와 음...,5
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 ^^ 맛있게 드시고요 환절기에 감기조심하시고요\n앞으로도 만족을 ...,5
4,모든 반찬이 다 맛있습니다!!!,주문 감사드립니다 ^^ 이번 주 월요일도 화이팅입니다~\n미세먼지 조심하시고 감기도...,5


##0_증강 데이터 추가

###1_ 0905 공유 / train_over.csv

In [ ]:
train_over = pd.read_csv(f"{DATA_PATH}train_over.csv")
train_over

,review,reply,star
0,아니 그런데 배달이 20분 넘게 안 와서 전화해서 물었더니 배달 기사님이 비가 와서...,저희가 미처 생각하지 못한 부분이 있기 때문에 다시 한번 감사의 마음을 전하고 싶습니다.,1
1,양이 2만원씩이나하는데 장난으로하는것도 아니고 진짜 이거 2만원주고 사서 먹을 정도...,저희 1988응답하라추억의옛날도시락은 언제나 약속 드린 정량을 지키기 위해 노력하고...,1
2,커피가 배달되기를 기다린 지 25분이 지나자 커피 배달 예정 시간은 이미 커피를 만...,한파로 길이 꽁꽁 얼어 고객님들이 불편을 드려 죄송했는데 주문해 주셔서 감사합니다.,1
3,김이 많아서 그런것도 있고 단무짠지 김이 많이 없어서 그런지도 모르겠지만 밥이 쉬었...,이런 댓글은 본인 가족이나 지인에게 하는데 본인이 아니더라도 주변 사람이 해야 할 ...,1
4,처음 먹었을 때 너무 맛있게 먹었던 기억에 다시 찾아 주문하려고 잠실점에 갔는데 역...,잠실점에서 기분이 상하셨었는데 저희가 다시 실수를 한 것 같은데 다시 한번 연락드리...,1
...,...,...,...
721,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4
722,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4
723,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4
724,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4


In [ ]:
train_over.columns=['고객리뷰','사장댓글','별점']

- 증강 데이터 별점

In [ ]:
for i in range(1, 6):
    num_rating = train_over[train_over['별점'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / train_over.shape[0] * 100}%")

별점 1점 개수: 90, 12.396694214876034%
별점 2점 개수: 61, 8.402203856749312%
별점 3점 개수: 153, 21.074380165289256%
별점 4점 개수: 422, 58.1267217630854%
별점 5점 개수: 0, 0.0%


1_ 기존데이터 + 증강데이터(train_over)

In [ ]:
train = pd.concat([df_train,train_over]).reset_index(drop=True)
train

,고객리뷰,사장댓글,별점
0,비 오는데도 배달 빨리 해주시고 감사합니다!!\n맛있어요옹,"안녕하세요 마라하오죽전점 입니다.\n즐거운 시간 보내시고, \n소중한 리뷰 작성해주...",5
1,2인세트인데 양도 엄청많아서 좋았어요!\n맛있게 잘 먹었습니당 ㅎㅎ,놀부부대찌개를 이용해주셔서 감사합니다 ^^\n맛있는 음식을 넉넉하게 드실수 있도록\...,4
2,너무맛있어요!! 자주시켜먹게되네요ㅎㅎ,은근히 당기는 맛들로 준비하고 있습니다\n직원들의 입맛을 통과한 맛있는 디져트와 음...,5
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 ^^ 맛있게 드시고요 환절기에 감기조심하시고요\n앞으로도 만족을 ...,5
4,모든 반찬이 다 맛있습니다!!!,주문 감사드립니다 ^^ 이번 주 월요일도 화이팅입니다~\n미세먼지 조심하시고 감기도...,5
...,...,...,...
20719,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4
20720,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4
20721,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4
20722,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4


In [ ]:
for i in range(1, 6):
    num_rating = train[train['별점'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / train.shape[0] * 100}%")

별점 1점 개수: 179, 0.8637328701022968%
별점 2점 개수: 131, 0.6321173518625748%
별점 3점 개수: 334, 1.6116579810847327%
별점 4점 개수: 1039, 5.013510905230651%
별점 5점 개수: 19041, 91.87898089171973%


### 2_0907 공유 / train_over_0905.csv (별점1~4 데이터 paraphrase한 데이터)

In [103]:
train_over_0905 = pd.read_csv(f"{DATA_PATH}train_over_0905.csv")
train_over_0905

,review,reply,star
0,아니 그런데 배달이 20분 넘게 안 와서 전화해서 물었더니 배달 기사님이 비가 와서...,응대 방법을 다시 한번 점검해 보겠습니다 고객님이 불편을 드려 죄송합니다 다음에 또...,1
1,커피가 배달되기를 기다린 지 25분이 지나자 커피 배달 예정 시간은 이미 커피를 만...,추운 날씨로 인해 길이 꽁꽁 얼어 고객님들이 많이 힘들어 하셨는데 다시 주문해 주셔...,1
2,이 씨 부부의 경우 임신 8개월째로 배 속 아기가 근심되어 허리를 약간 굽히고 한 ...,사이드만 잔뜩 시키고 1+1 사이드만 시키는 그런 스타일이에요.,1
3,김이 많아서 그런것도 있고 단무짠지 김이 많이 없어서 그런지도 모르겠지만 밥이 쉬었...,이런 댓글은 본인 가족이나 지인에게 하는데 본인이 아니더라도 주변 사람이 해야 할 ...,1
4,처음 먹었을 때 너무 맛있게 먹었던 기억에 다시 찾아 주문하려고 잠실점에 갔는데 역...,잠실에서 기분이 나쁘셨었는데 저희가 또 실수를 했네요 사정을 다시 말씀 드리겠습니다.,1
...,...,...,...
1191,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4
1192,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해본 결과 직...,4
1193,진짜 맛있어요 특히 간장 양념이 너무 달지 않고 새콤해서 더 맛있게 느껴져요,많은 사랑 주셔서 감사드립니다 감사합니다 말씀 드릴 것이 많은데도 불구하고 부족한 ...,4
1194,간편하게 끓여서 먹기도 하고 찌개 끓이는 것도 귀찮아도 간편해서 좋아요.,NaN,4


- 결측치 제거

In [104]:
train_over_0905.isnull().sum()

review    0
reply     2
star      0
dtype: int64

In [105]:
train_over_0905 = train_over_0905.dropna(subset=['reply'])

In [106]:
train_over_0905.isnull().sum()

review    0
reply     0
star      0
dtype: int64

In [107]:
for i in range(1, 6):
    num_rating = train_over_0905[train_over_0905['star'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / train_over_0905.shape[0] * 100}%")

별점 1점 개수: 111, 9.296482412060302%
별점 2점 개수: 87, 7.2864321608040195%
별점 3점 개수: 227, 19.01172529313233%
별점 4점 개수: 769, 64.40536013400336%
별점 5점 개수: 0, 0.0%


2_기존데이터 + 증강데이터(train_over_0905)

In [108]:
train_over_0905.columns=['고객리뷰','사장댓글','별점']

In [109]:
train = pd.concat([df_train,train_over_0905]).reset_index(drop=True)
train

,고객리뷰,사장댓글,별점
0,비 오는데도 배달 빨리 해주시고 감사합니다!!\n맛있어요옹,"안녕하세요 마라하오죽전점 입니다.\n즐거운 시간 보내시고, \n소중한 리뷰 작성해주...",5
1,2인세트인데 양도 엄청많아서 좋았어요!\n맛있게 잘 먹었습니당 ㅎㅎ,놀부부대찌개를 이용해주셔서 감사합니다 ^^\n맛있는 음식을 넉넉하게 드실수 있도록\...,4
2,너무맛있어요!! 자주시켜먹게되네요ㅎㅎ,은근히 당기는 맛들로 준비하고 있습니다\n직원들의 입맛을 통과한 맛있는 디져트와 음...,5
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 ^^ 맛있게 드시고요 환절기에 감기조심하시고요\n앞으로도 만족을 ...,5
4,모든 반찬이 다 맛있습니다!!!,주문 감사드립니다 ^^ 이번 주 월요일도 화이팅입니다~\n미세먼지 조심하시고 감기도...,5
...,...,...,...
21187,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,맛있는 식사가 되셨기를 바랍니다 맛있는 식사 이용해 주셔서 감사합니다,4
21188,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행 업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락주시면 ...,4
21189,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4
21190,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해본 결과 직...,4


In [110]:
train[train.duplicated(subset='고객리뷰', keep=False)]

,고객리뷰,사장댓글,별점
12,잘 먹었습니다~~~,이용해주시고 맛있게드셔주셔서 감사드립니다! 즐거운 주말 보내세요!~^^♡,5
13,맛있어요 감사합니다,소중한 리뷰 남겨주셔서 감사드려요~ ^^\n다음에도 맛있는 보쌈 드시고 싶으실때 또...,5
21,맛나게 잘 먹었습니다~~,안녕하세요(●'◡'●) 소중한 고객님!\n\n소중한 후기 감사드립니다❤\n사진과 함...,5
59,오늘도 잘 먹었습니다,잊지 않고 소중한 포토리뷰 남겨주셔서 정말 감사합니다~!🥰🥰\n\n저희 사진은 아니...,5
60,맛있게 잘 먹었습니다!!,안녕하세요! 푸짐하구 맛있는 헬로팬입니다 :) \n맛있게 잘 드셔주셔서 감사해여! ...,5
...,...,...,...
21157,맛있게 잘 먹었습니다 감사합니다 맛있게 잘,그럼에도 맛있게 잘 먹어 주셔서 감사합니다. 그럼에도 배달업체에 요청사항이 있는데 ...,4
21180,맛있게 잘 먹었습니다.,언제나 맛있고 넉넉한 음식을 준비하는 데 최선을 다해 고객만족을 위해 노력하겠습니다.,4
21188,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행 업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락주시면 ...,4
21189,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4


In [111]:
for i in range(1, 6):
    num_rating = train[train['별점'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / train.shape[0] * 100}%")

별점 1점 개수: 200, 0.9437523593808984%
별점 2점 개수: 157, 0.7408456021140053%
별점 3점 개수: 408, 1.9252548131370328%
별점 4점 개수: 1386, 6.540203850509626%
별점 5점 개수: 19041, 89.84994337485843%


##1_특수문자 제거

In [112]:
review_list = []
for review in train['고객리뷰']:
    if isinstance(review, str):
        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['고객리뷰'] = review_list
train.head()

,고객리뷰,사장댓글,별점
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다.\n즐거운 시간 보내시고, \n소중한 리뷰 작성해주...",5
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 ^^\n맛있는 음식을 넉넉하게 드실수 있도록\...,4
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다\n직원들의 입맛을 통과한 맛있는 디져트와 음...,5
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 ^^ 맛있게 드시고요 환절기에 감기조심하시고요\n앞으로도 만족을 ...,5
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 ^^ 이번 주 월요일도 화이팅입니다~\n미세먼지 조심하시고 감기도...,5


In [113]:
reply_list = []
for reply in train['사장댓글']:
    if isinstance(reply, str):
        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['사장댓글'] = reply_list
train.head()

,고객리뷰,사장댓글,별점
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5


##2_띄어쓰기/맞춤법

In [19]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-ndj8yxtq
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-ndj8yxtq
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=aef66df0d9bc215929fc9514dc5aa51d5bbc208f9cfa19e9ed4d6a12454a4a90
  Stored in directory: /tmp/pip-ephem-wheel-cache-px8n2jho/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [20]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
corrected_review = []
for review in tqdm(train['고객리뷰']):
    tmp = correct_spell(review)
    corrected_review.append(tmp)

  0%|          | 0/20724 [00:00<?, ?it/s]

In [ ]:
train["corrected_review"] = corrected_review

In [ ]:
corrected_reply = []
for review in tqdm(train['사장댓글']):
    tmp = correct_spell(review)
    corrected_reply.append(tmp)

  0%|          | 0/20724 [00:00<?, ?it/s]

In [ ]:
train["corrected_reply"] = corrected_reply

In [ ]:
train

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔..."
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~
...,...,...,...,...,...
20719,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4,맛있게 잘 먹었습니다.,더 노력하는 보둘 미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 ...
20720,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부으시면 국물이 흘러내리기 때문에 혹시라도 모자라면 댁에서 생...
20721,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...
20722,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다


In [ ]:
train.to_csv(f"{DATA_PATH}요기요_맞춤법수정_0906.csv",index=False) # 저장

### 문장 추출 / 엑셀 저장

In [ ]:
# 패턴을 정의합니다. 여러 가지 구분 기호와 줄바꿈 문자를 포함합니다.
sentence_pattern = re.compile(r'[^.!?]*[.!?]|[^.!?]+$') # $로 구분기호 없이 끝나는 경우 포함   # 구분기호 없는 경우 포함 문장 분리 버전. 이 버전 or 아래 간단한 버전 쓰면될 듯
yogiyo_reviews_sentence = []
yogiyo_replies_sentence = []

for review in train["corrected_review"]:  # text는 문자열 리뷰들의 리스트입니다
    if isinstance(review, str):  # 리뷰가 문자열인지 확인합니다
        sentences = sentence_pattern.findall(review)
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        yogiyo_reviews_sentence.append(sentences)
    else:
        review = '리뷰없음'
        yogiyo_reviews_sentence.append(review) # 리뷰 없는 경우도 갯수, 순서 맞추기 위해 넣어줌
        print(review)

for review in train["corrected_reply"]:  # text는 문자열 리뷰들의 리스트입니다
    if isinstance(review, str):  # 리뷰가 문자열인지 확인합니다
        sentences = sentence_pattern.findall(review)
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        yogiyo_replies_sentence.append(sentences)
    else:
        review = '리뷰없음'
        yogiyo_replies_sentence.append(review) # 리뷰 없는 경우도 갯수, 순서 맞추기 위해 넣어줌
        print(review)

- 엑셀 저장

In [ ]:
tmp = pd.DataFrame.from_records(yogiyo_reviews_sentence)
tmp.to_excel('요기요_고객리뷰_맞춤법수정.xlsx')

In [ ]:
tmp = pd.DataFrame.from_records(yogiyo_replies_sentence)
tmp.to_excel('요기요_사장댓글_맞춤법수정.xlsx')

##3_고객ID -> 고객님

In [ ]:
train = pd.read_csv(f"{DATA_PATH}요기요_맞춤법수정_0906.csv")
train

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔..."
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~
...,...,...,...,...,...
20719,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4,맛있게 잘 먹었습니다.,더 노력하는 보둘 미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 ...
20720,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부으시면 국물이 흘러내리기 때문에 혹시라도 모자라면 댁에서 생...
20721,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...
20722,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다


In [ ]:
train['corrected_review'] = train['corrected_review'].astype(str)
train['corrected_reply'] = train['corrected_reply'].astype(str)

In [ ]:
train['corrected_reply'] = train['corrected_reply'].str.replace(r'\w+\s* 님', '고객님', regex=True)

In [ ]:
train["corrected_reply"][33]

'고객님 주문해 주셔서 고맙습니다 항상 신선 달달한 과일과 디저트 보내드리기 위해 꾸준히 신경 쓸게 욥 오늘도 편안하고 즐거운 하루 보내세요~!'

In [ ]:
train["corrected_reply"][679] # 배달기사님 유지

'너무너무 죄송합니다 배달에 문제가 생겼었나 보네요 배달기사님께 서 도 말씀이 없으셔서 저는, 아무런 전달을 못 받아서 제가 정확한 내용을 전혀 모르고 있네요 앞으로 배달 문제가 생기면 언제든지 연락 부탁드리며. 빠르게 해결하겠습니다 번거롭게 해드려 너무 죄송합니다 ~'

In [ ]:
train["corrected_reply"][1445] # 부모님 유지

'부모님도 아이들도 모두 만족해 주셔서 정말 다행입니다 맛있게 드셔주시고 소중한 리뷰 달아주셔서 정말 감사드립니다! '

##4_가게명 키워드 컬럼화

In [124]:
store_list = yogiyo['가게이름']
store_list = (store_list.str.split('-')).str.get(0)
store_list

0        1988응답하라추억의옛날도시락
1        1988응답하라추억의옛날도시락
2        1988응답하라추억의옛날도시락
3        1988응답하라추억의옛날도시락
4        1988응답하라추억의옛날도시락
               ...       
24993             놀부 부대찌개
24994             놀부 부대찌개
24995             놀부 부대찌개
24996             놀부 부대찌개
24997             놀부 부대찌개
Name: 가게이름, Length: 24998, dtype: object

- 띄어쓰기한 가게명

In [125]:
tmp = list(set(store_list.replace("[^가-힣a-zA-Z0-9 ]","",regex=True)))

In [126]:
tmp2 = []
for name in tmp:
    hanspell_sent = spell_checker.check(name)
    tmp2.append(hanspell_sent.checked)

tmp2

['써브웨이',
 '피자 이탈리',
 '최가네 한 쌈',
 '덮덮밥',
 '폴트 버거',
 '39도시락',
 '탕화 쿵후 마라탕 가오점',
 '육회 연어',
 '피자 보이 시나',
 '공주 닭발',
 '롱 쓰 부대찌개',
 '병천 토속 순대',
 '나인 곱창',
 '홍싸롱',
 '춘리 마라탕',
 '인생 제육',
 '마라의신마라탕마라샹궈',
 '상무 초밥',
 '랜돌프 뉴욕 페퍼로니',
 '당치 땡',
 '올 데이 파스타',
 '안녕 닭',
 '큰아들 백순대',
 '알촌',
 '순 살만 공격',
 '빅스타 피자',
 '킹 프레 소빅 와 플',
 '더 팔당 매운 오징어 닭발 김치찜',
 '홍대 개미 ',
 '밈이 피자',
 '닭 강점',
 '땡초 곱창 막창',
 '신 자매 김치찜 김치찌개',
 '카페 봄봄',
 '이토야',
 '삽교 원조 두리 곱창',
 '모 모 다코야키 카페',
 '이끌림 마라탕',
 '저팔계 중국 마라탕 덮밥요리 전문점',
 '오늘은 분식',
 '후스 폭립',
 '빕스얌딜리버리',
 '한식 세 끼 1인 김치찜 김치찌개',
 '보도로 미역',
 '고기 듬뿍 미트 박스',
 '수락',
 '돈가스 회관',
 '울진 죽변항',
 '국수나무',
 '윤희 횟집',
 '플러스 82',
 '우시장 국밥',
 '피자 프라 텔로',
 '모어 덴버 터',
 '비 오키친',
 '행복한 찜닭',
 '홍수 족발',
 '수비자',
 '워커스 커피 로프터스',
 '코알라 파이',
 '헬로 팬 닭갈비 마약 볶음밥',
 '고씨네 카레',
 '청년 치킨',
 '마라 쿡',
 '초밥 대통령',
 '여왕의 브런치',
 '시민 보쌈족발 감자탕',
 '구래 상회',
 '냉면숯불고기전문점줄벼락',
 '우리 집 반찬도시락',
 '자 성당',
 '피자 스웨이',
 '올 데이 케밥 샐러드',
 '유쾌한 꼬막 한 끼 끝판왕',
 '놀부 부대찌개',
 '도야 짬뽕 계림점',
 'GTS 버거',
 '최고 날제육 명가',
 '고기 혁명',
 '본점 장충 왕족발보쌈',
 '탐나는 피자',
 '명작 

In [127]:
store_list = store_list.unique().tolist()
store_list

['1988응답하라추억의옛날도시락',
 '워커스커피 로스터스',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕&마라샹궈',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜&김치찌개',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨족발보쌈',
 '육회&연어',
 '갈비민족',
 '구구족',
 '피자보이시나',
 '왕빈자삼파전',
 '행복한찜닭',
 '스시사소우',
 '저팔계&중국&마라탕&덮밥&요리&전문점',
 '명작파스타',
 '올데이파스타',
 '대한냉면',
 '인생제육',
 '고기듬뿍미트박스',
 '피자스웨그',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '더팔당매운오징어&닭발 김치찜',
 '우리집반찬도시락',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비&마약볶음밥',
 '대찌 대파생고기김치찌개&정육왕국물두루치기',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '1989마라탕',
 '미친피자',
 '천년닭강정',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은 분식',
 '이끌림마라탕',
 '탐나는피자',
 '고기혁명',
 '라화방마라탕',
 '전금례닭볶음탕',
 '큰아들백순대',
 '공주닭발',
 '신자매김치찜&김치찌개',
 '이태리면가게',
 '폴트버거',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부덮밥',
 '카산도',
 '부성초밥',
 '닭장수섭삼계탕&전기구이통닭',
 '깨돌이김밥',
 '울진죽변항'

- 띄어쓰기 있을 경우 띄어쓰기 없앤 가게명(ex)놀부부대찌개

In [128]:
tmp3 = [''.join(store.split()) for store in store_list]

In [129]:
short_name = []
for name in store_list:
    for length in [2, 3, 4]:
        cut = name[:length]
        short_name.append(cut)
    if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
        cut = name[:6]
        short_name.append(cut)

short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
short_name = list(set(short_name))
short_name = sorted(short_name, reverse=True, key=len)
print(len(short_name), short_name)

428 ['1988응답', '1989마라', '39도시락', '7번가피자', '봉명동내', '춘리마라', '꼬알라파', '야키토리', '잭아저씨', '부대찌개', '국수나무', '비오키친', '홍스족발', '인생아구', '올데이파', '대찌 대', '고기혁명', '이화수육', '쇼부덮밥', '플러스8', '육회&연', '갈비민족', 'GTS버', '부성초밥', '미친피자', '뜸들이다', '놀부보쌈', '탐나는피', '저팔계&', '우시장국', '공주닭발', '탕화쿵푸', '인생제육', '피자보이', '더팔당매', '토핑폭탄', '홍대개미', '39도시', '조형훈족', '한식세끼', '이토우야', '도우라이', '전금례닭', '코브라독', '피제이피', '손수경의', '큰아들백', '냉면&숯', '청년치킨', '랜돌프뉴', '고씨네카', '진지덮밥', '시민보쌈', '짱닭치킨', '깨돌이김', '걸작떡볶', '스시사소', '왕빈자삼', '이끌림마', '모모타코', '본점 장', '명작파스', '대한냉면', '손이가시', '최가네한', '삽교원조', '무국적식', '유쾌한꼬', '더티베지', '폴트버거', '승도리네', '송파만다', '상무초밥', '호랑이초', '킹프레소', '이배달 ', '우리집반', '라화방마', '빕스얌딜', '카페봄봄', '후스폭립', '오늘은 ', '수상한삼', '신자매김', '배달의쌀', '황궁쟁반', '구래상회', '닭장수섭', '이태리면', '피자스웨', '돈까스회', '미락칼국', '빅스타피', '초밥대통', '짚신스시', 'OK반점', '고기듬뿍', '땅끝수산', '감성커피', '청춘식당', '여왕의브', '텐텐마라', '창타이누', '마라하오', '수제죽전', '병천토속', '놀부 부', '땡초곱창', '혼밥대왕', '도야짬뽕', '써브웨이', '팔도한마', '7번가피', '나인곱창', '울진죽변', '피자이탈', '최고남제', '순살만공', '안녕,닭', '보돌미역', '띵하오차', '마라의신', '윤희횟집', '올데이케

In [130]:
short_name

['1988응답',
 '1989마라',
 '39도시락',
 '7번가피자',
 '봉명동내',
 '춘리마라',
 '꼬알라파',
 '야키토리',
 '잭아저씨',
 '부대찌개',
 '국수나무',
 '비오키친',
 '홍스족발',
 '인생아구',
 '올데이파',
 '대찌 대',
 '고기혁명',
 '이화수육',
 '쇼부덮밥',
 '플러스8',
 '육회&연',
 '갈비민족',
 'GTS버',
 '부성초밥',
 '미친피자',
 '뜸들이다',
 '놀부보쌈',
 '탐나는피',
 '저팔계&',
 '우시장국',
 '공주닭발',
 '탕화쿵푸',
 '인생제육',
 '피자보이',
 '더팔당매',
 '토핑폭탄',
 '홍대개미',
 '39도시',
 '조형훈족',
 '한식세끼',
 '이토우야',
 '도우라이',
 '전금례닭',
 '코브라독',
 '피제이피',
 '손수경의',
 '큰아들백',
 '냉면&숯',
 '청년치킨',
 '랜돌프뉴',
 '고씨네카',
 '진지덮밥',
 '시민보쌈',
 '짱닭치킨',
 '깨돌이김',
 '걸작떡볶',
 '스시사소',
 '왕빈자삼',
 '이끌림마',
 '모모타코',
 '본점 장',
 '명작파스',
 '대한냉면',
 '손이가시',
 '최가네한',
 '삽교원조',
 '무국적식',
 '유쾌한꼬',
 '더티베지',
 '폴트버거',
 '승도리네',
 '송파만다',
 '상무초밥',
 '호랑이초',
 '킹프레소',
 '이배달 ',
 '우리집반',
 '라화방마',
 '빕스얌딜',
 '카페봄봄',
 '후스폭립',
 '오늘은 ',
 '수상한삼',
 '신자매김',
 '배달의쌀',
 '황궁쟁반',
 '구래상회',
 '닭장수섭',
 '이태리면',
 '피자스웨',
 '돈까스회',
 '미락칼국',
 '빅스타피',
 '초밥대통',
 '짚신스시',
 'OK반점',
 '고기듬뿍',
 '땅끝수산',
 '감성커피',
 '청춘식당',
 '여왕의브',
 '텐텐마라',
 '창타이누',
 '마라하오',
 '수제죽전',
 '병천토속',
 '놀부 부',
 '땡초곱창',
 '혼밥대왕',
 '도야짬뽕',
 '써브

In [ ]:
# # short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
# drop_name = ['OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교','손수','송파','쇼부','수상','수제','시민','오늘','오늘은 ','우리','울진','이끌','이끌림','인생','직화','진지','천년','청년','청춘','최고','큰아','큰아들','탐나','토핑','행복','행복한','헬로','홍대','육회','배달의','야키토리','호랑이','우리집','한식','부대찌개','피자','스시','여왕','초밥','냉면','본점','갈비','카페','마라','안녕','유쾌']
# short_name = [col for col in short_name if col not in drop_name]
# print(len(short_name), short_name)

367 ['마라의', '피제이피', '오늘은', '인생아구', '더티', '자성당', '뜸들이', '더티베', '윤희횟', '카페봄봄', '돈까스', '모모', '미락칼국', '수피', '라화', '모어댄', '부성초', '피자스웨', '후스폭', '피자프', '갈비민', '손수경의', '어시', '저팔', '피자이', '본점 장', '당치', '당치땡', '빅스', '냉면&숯', '윤희', '최고남', '나인곱', '써브웨', '부산오', '텐텐마라', '우시', '고기잔뜩', '걸작떡볶', '폴트버', '올데이', '깨돌이김', '최가', '라화방마', '피자보', '왕빈', '마라쿡', '전금례닭', '송파만다', '팔도한', '혼밥대', '피제', '청춘식', '피자스', '짜글', '빽다방', '구래상회', '조형', '왕빈자삼', '부성초밥', '놀부', '더팔', '승도', '카페봄', '후스', '탕화쿵', '혼밥대왕', '홍스족', '롱쓰부대', '이화', '대찌 ', '나인', '천년닭', '봉명', '미락칼', '39도시', '탐나는', '모어댄버', '덮덮밥', '뜸들이다', '짱닭치', '피자이탈', '모모타', '땡초곱', '과일에', '송파만', '써브웨이', '떡깨비', '워커스', '7번가피', '도우라', '승도리', '저팔계&', '유쾌한꼬', '워커', '부대찌', '병천토', '국수나', '짜글이', '신자매김', '감성커', '짚신스시', '초밥대', '황궁', '삼다수', '대찌 대', '창타이', '마라하', '랜돌프', '랜돌프뉴', '수상한삼', '빕스얌딜', '청춘식당', '라화방', '도야짬뽕', '순살만공', '신자', '혼밥', '스시리', '야키토', '대한냉', '이토', '명작파스', '잭아저', '공주닭', '초밥대통', '쇼부덮', '공주닭발', '구래상', '코브', '짚신스', '무국적', '스시사소', '땅끝수', '창타', '1988응답', '저팔계', '이화수육', '구래', '울진죽', 

In [131]:
# short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
drop_name = ['야키토리','부대찌개','오늘은 ','고기잔뜩','돈까스','탐나는','안녕,','닭강정','행복한','호랑이','어시장','이태리','배달의','이끌림','큰아들','유쾌한','오늘은','수상한','짜글이','본점 ','플러스','과일에','국수나','우리집','인생아','손이가','삼다수','행복','자성','배달','청년','육회','토핑','이화','최고','땡초','직화','짚신','7번','울진','부산','초밥','수제','수상','혼밥','걸작','냉면','카페','진지','과일','오늘','공주','피자','홍대','국수','부대','손수','김포','팔도','한식','송파','안녕','인생','갈비','시민','감성','미친','고기','우리','도우','땅끝','여왕','OK','대한','헬로','청춘','수락','순살','본점','유쾌''삼다','무국','이끌','천년','명작','마라','병천','청춘','스시','쇼부']
short_name = [col for col in short_name if col not in drop_name]
print(len(short_name), short_name)

342 ['1988응답', '1989마라', '39도시락', '7번가피자', '봉명동내', '춘리마라', '꼬알라파', '잭아저씨', '국수나무', '비오키친', '홍스족발', '인생아구', '올데이파', '대찌 대', '고기혁명', '이화수육', '쇼부덮밥', '플러스8', '육회&연', '갈비민족', 'GTS버', '부성초밥', '미친피자', '뜸들이다', '놀부보쌈', '탐나는피', '저팔계&', '우시장국', '공주닭발', '탕화쿵푸', '인생제육', '피자보이', '더팔당매', '토핑폭탄', '홍대개미', '39도시', '조형훈족', '한식세끼', '이토우야', '도우라이', '전금례닭', '코브라독', '피제이피', '손수경의', '큰아들백', '냉면&숯', '청년치킨', '랜돌프뉴', '고씨네카', '진지덮밥', '시민보쌈', '짱닭치킨', '깨돌이김', '걸작떡볶', '스시사소', '왕빈자삼', '이끌림마', '모모타코', '본점 장', '명작파스', '대한냉면', '손이가시', '최가네한', '삽교원조', '무국적식', '유쾌한꼬', '더티베지', '폴트버거', '승도리네', '송파만다', '상무초밥', '호랑이초', '킹프레소', '이배달 ', '우리집반', '라화방마', '빕스얌딜', '카페봄봄', '후스폭립', '수상한삼', '신자매김', '배달의쌀', '황궁쟁반', '구래상회', '닭장수섭', '이태리면', '피자스웨', '돈까스회', '미락칼국', '빅스타피', '초밥대통', '짚신스시', 'OK반점', '고기듬뿍', '땅끝수산', '감성커피', '청춘식당', '여왕의브', '텐텐마라', '창타이누', '마라하오', '수제죽전', '병천토속', '놀부 부', '땡초곱창', '혼밥대왕', '도야짬뽕', '써브웨이', '팔도한마', '7번가피', '나인곱창', '울진죽변', '피자이탈', '최고남제', '순살만공', '안녕,닭', '보돌미역', '띵하오차', '마라의신', '윤희횟집', '올데이케', '과일에반', '피자프라', '천년닭강

In [132]:
store = store_list + tmp2 + tmp3 + short_name

In [133]:
store_list = list(set(store))
store_list

['빽다',
 '덮덮밥',
 '피자이탈리',
 '최가네한쌈',
 '상무',
 '탕화 쿵후 마라탕 가오점',
 '이토',
 '떡 형',
 '피자보이시나',
 '봉명동내',
 '롱쓰부대찌개',
 '봉명동 내 커피',
 '본점 장충 왕족발보쌈',
 '병천 토속 순대',
 '부산오',
 '코브',
 '구래 상회',
 '춘리마라',
 '창타이',
 '맘스터치대덕테크노벨리점',
 '땅끝 수산',
 '공주 닭발',
 '큰아들백순대',
 '꼬알라파',
 '1988응답하라 추억의 옛날 도시락',
 '돈가스 회관',
 '홍스족',
 '빅스타피자',
 '비 오키친',
 '탕화쿵',
 '이토야',
 '잭아저씨',
 '밈피자',
 '닭강점',
 '밈이 피자',
 '명작파',
 '텐텐마',
 '깨돌이 김밥',
 '이화수 육개장',
 '대치 대파 생고기 김치찌개 정육 왕국 물 두루치기',
 '빕스얌딜리버리',
 '장닭 치킨',
 '모모타코야키&카페',
 '명작 파스타',
 '플러',
 '잭아',
 '울진죽변항',
 '국수나무',
 '더팔당매운오징어&닭발김치찜',
 '창타',
 '피자프라텔로',
 '워커스',
 '비오키친',
 '행복한찜닭',
 '홍스족발',
 '텐텐 마라탕',
 '워커스커피 로스터스',
 '피자 스웨이',
 'GTS 버거',
 '인생아구',
 '승도',
 '마라쿡',
 '카페봄',
 '여왕의브런치',
 '플러스 82',
 '깨돌',
 '과일에 반하다',
 '저팔계&중국&마라탕&덮밥&요리&전문점',
 '올데이파',
 '이배달마약삼겹떡볶이상당구점',
 'OK반',
 '떡 깨비',
 '전금',
 '신 자매 김치찜 김치찌개',
 '고 돼지',
 '냉면&',
 'GTS버거',
 '대찌 대',
 '고기혁명',
 '짱닭',
 '탐나는피자',
 '시민보',
 '더티베',
 '돈까',
 '비오',
 '고기잔뜩짜글이찌개&부대찌개',
 '걸작떡볶이',
 '코브라독스',
 '토핑폭',
 'GTS',
 '도야짬',
 '토핑 폭탄 김치찜 김치찌개',
 '감성커',
 '팔도한마당순대',
 '우시'

- '당신은 지금 치킨이 땡긴다' 추가

In [134]:
store_list.append('당신은 지금 치킨이 땡긴다')

In [135]:
store_list.append('당신은 지금 치킨이 당긴다')

In [136]:
store_list.append('마라 하오')

In [ ]:
train['store'] = ""
for name in store_list:
    train[name] = train['corrected_reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
    train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
train = train.drop(columns=store_list) # 각 가게명 컬럼 삭제

train.head()

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food,review,sentiment,score,kiwi_reviews
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",마라 하오,죽전점,마라,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,1,0.761931,"[배달, 감사, 맛있]"
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,"놀부,부대찌,대찌",,부대찌개,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,1,0.586811,"[세트, 많, 좋, 맛있]"
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,,,,너무 맛있어요! 자주 시켜 먹게 되네요,1,0.779962,[맛있]
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,,,,항상 맛있어요 서비스도 최고,1,0.801292,"[맛있, 서비스, 최고]"
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,,,파이,모든 반찬이 다 맛있습니다!,1,0.762959,"[반찬, 맛있]"


In [ ]:
train['store'][1245]

'랜돌프,랜돌,랜돌프 뉴욕 페퍼로니'

In [ ]:
train['store'][1264]

'놀부'

##5_지점명 키워드 컬럼화

In [139]:
store_list = yogiyo['가게이름']
branch_list = (store_list.str.split('-')).str.get(1)
branch_list = branch_list.unique()
branch_list

array(['방이점', nan, '가락점', '헬리오시티점', '석촌점', '송파점', '삼전점', '본점', '미아본점',
       '성신여대역점', '대학로점', '성북점', '동대문구점', '북서울꿈의숲점', '혜화막내딸지점', '부천점',
       '부평점', '부천부평점', '본가진죽', '신중동점', '부천시청점', '도촌점', '야탑점', '판교점',
       '수제돈까스&파스타', '정자본점', '미금본점', '미금점', '분당정자점', '분당미금역점', '죽전점',
       '용인수지점', '서현본점', '분당점', '용암점', '대전4호점', '신탄진점', '신일점', '목동충촌점',
       '대전동구점', '우송대점', '대전역점', '대흥문창점', '서대전점', '유성봉명점', '괴정동', '대전 관평점',
       '대전전민점', '대전한남대점', '충남대점', '유성점', '광주백운점', '조대점', '광주본점', '광주광천점',
       '북구(전대)점', '광주직영점', '광주우산점'], dtype=object)

In [140]:
branch_list = np.delete(branch_list, [1, 24])
branch_list

array(['방이점', '가락점', '헬리오시티점', '석촌점', '송파점', '삼전점', '본점', '미아본점',
       '성신여대역점', '대학로점', '성북점', '동대문구점', '북서울꿈의숲점', '혜화막내딸지점', '부천점',
       '부평점', '부천부평점', '본가진죽', '신중동점', '부천시청점', '도촌점', '야탑점', '판교점',
       '정자본점', '미금본점', '미금점', '분당정자점', '분당미금역점', '죽전점', '용인수지점', '서현본점',
       '분당점', '용암점', '대전4호점', '신탄진점', '신일점', '목동충촌점', '대전동구점', '우송대점',
       '대전역점', '대흥문창점', '서대전점', '유성봉명점', '괴정동', '대전 관평점', '대전전민점',
       '대전한남대점', '충남대점', '유성점', '광주백운점', '조대점', '광주본점', '광주광천점',
       '북구(전대)점', '광주직영점', '광주우산점'], dtype=object)

In [141]:
branch_list = branch_list.tolist()

In [142]:
tmp2 = []
for name in branch_list:
    hanspell_sent = spell_checker.check(name)
    tmp2.append(hanspell_sent.checked)

tmp2

['방이점',
 '가락점',
 '헬리오 시티점',
 '석촌점',
 '송파점',
 '삼전점',
 '본점',
 '미아 본점',
 '성신여대역점',
 '대학로점',
 '성북점',
 '동대문구점',
 '북서울꿈의 숲점',
 '혜화 막내딸 지점',
 '부천점',
 '부평점',
 '부천 부평점',
 '본 가진 죽',
 '신중동점',
 '부천 시청점',
 '도촌점',
 '야탑점',
 '판교점',
 '정자본점',
 '미금 본점',
 '미금점',
 '분당 정자점',
 '분당 미금역점',
 '죽전점',
 '용인 수지점',
 '서현 본점',
 '분당점',
 '용암점',
 '대전 4호점',
 '신탄진점',
 '신일점',
 '목동충촌점',
 '대전 동구점',
 '우송대점',
 '대전역점',
 '대흥 문창점',
 '서대전점',
 '유성 봉명점',
 '괴정동',
 '대전 관평점',
 '대전 전민점',
 '대전 한남대점',
 '충남대점',
 '유성점',
 '광주백운점',
 '조대점',
 '광주 본점',
 '광주 광천점',
 '북구(전대) 점',
 '광주 직영점',
 '광주 우산점']

In [143]:
branch_list = branch_list + tmp2

In [144]:
branch_list = list(set(branch_list)) # 중복 제거
branch_list

['대전역점',
 '광주본점',
 '서대전점',
 '광주직영점',
 '대전4호점',
 '미아본점',
 '분당 미금역점',
 '미아 본점',
 '신일점',
 '광주백운점',
 '유성 봉명점',
 '우송대점',
 '북구(전대) 점',
 '신탄진점',
 '용암점',
 '분당점',
 '북서울꿈의 숲점',
 '혜화막내딸지점',
 '미금본점',
 '분당미금역점',
 '분당 정자점',
 '미금 본점',
 '괴정동',
 '충남대점',
 '헬리오시티점',
 '분당정자점',
 '대전 전민점',
 '서현본점',
 '조대점',
 '북서울꿈의숲점',
 '광주광천점',
 '본가진죽',
 '북구(전대)점',
 '성북점',
 '광주우산점',
 '대전 동구점',
 '유성봉명점',
 '광주 우산점',
 '동대문구점',
 '판교점',
 '유성점',
 '부평점',
 '부천부평점',
 '야탑점',
 '부천시청점',
 '용인수지점',
 '본점',
 '광주 본점',
 '대전동구점',
 '부천 시청점',
 '도촌점',
 '미금점',
 '목동충촌점',
 '대전 관평점',
 '신중동점',
 '대전전민점',
 '가락점',
 '부천점',
 '혜화 막내딸 지점',
 '석촌점',
 '용인 수지점',
 '성신여대역점',
 '부천 부평점',
 '대흥 문창점',
 '정자본점',
 '본 가진 죽',
 '서현 본점',
 '대학로점',
 '죽전점',
 '송파점',
 '대전한남대점',
 '대전 한남대점',
 '헬리오 시티점',
 '광주 직영점',
 '방이점',
 '삼전점',
 '광주 광천점',
 '대전 4호점',
 '대흥문창점']

In [ ]:
train['branch'] = train['corrected_reply'].apply(lambda x: next((branch for branch in branch_list if branch in x), " "))
train['branch']

0        죽전점
1           
2           
3           
4           
        ... 
20719       
20720       
20721       
20722       
20723       
Name: branch, Length: 20724, dtype: object

In [ ]:
train.head()

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",,죽전점
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,부대찌,
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,,
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,,
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,,


##6_음식 키워드 컬럼화

In [146]:
food_list = pd.read_csv(f"{DATA_PATH}food.csv").drop_duplicates(ignore_index = True)["대표식품명"]
food_list

0          케이크
1          치즈빵
2         카스텔라
3          츄러스
4         치아바타
        ...   
753      햄채소볶음
754    무생채(무만)
755     참나물겉절이
756     치커리겉절이
757       짜장소스
Name: 대표식품명, Length: 758, dtype: object

In [147]:
food_lst = []
for food in food_list:
    parts = re.split('/|-|\(', food)
    food_lst.append(parts[0])

In [148]:
food_lst

['케이크',
 '치즈빵',
 '카스텔라',
 '츄러스',
 '치아바타',
 '와플',
 '찰떡빵',
 '김밥',
 '덮밥',
 '보리밥',
 '볶음밥',
 '비빔밥',
 '삼각김밥',
 '수수밥',
 '알밥',
 '영양돌솥밥',
 '오므라이스',
 '자장밥',
 '잡곡밥',
 '잡채밥',
 '주먹밥',
 '짬뽕밥',
 '차조밥',
 '찰밥',
 '초밥',
 '카레라이스',
 '콩나물밥',
 '콩밥',
 '하이라이스',
 '현미밥',
 '회덮밥',
 '가래떡',
 '경단',
 '꿀떡',
 '모듬찰떡',
 '백설기',
 '송편',
 '시루떡',
 '약식',
 '인절미',
 '절편',
 '증편',
 '찹쌀떡',
 '도넛',
 '마늘빵',
 '머핀',
 '모닝빵',
 '모카빵',
 '베이글',
 '샌드위치',
 '소보로빵',
 '식빵',
 '초코소라빵',
 '크로와상',
 '파이',
 '페이스트리',
 '피자',
 '피자빵',
 '햄버거',
 '호떡',
 '간자장',
 '국수',
 '냉면',
 '떡국',
 '떡만두국',
 '라면',
 '만두',
 '만두국',
 '메밀국수',
 '수제비',
 '스파게티',
 '우동',
 '우동볶음',
 '월남쌈',
 '자장면',
 '짬뽕',
 '쫄면',
 '칼국수',
 '콩국수',
 '닭죽',
 '스프',
 '오리죽',
 '전복죽',
 '채소죽',
 '팥죽',
 '호박죽',
 '흑임자죽',
 '갈비탕',
 '감자국',
 '곰탕',
 '굴국',
 '김치국',
 '내장탕',
 '냉국',
 '달걀국',
 '달걀탕',
 '닭곰탕',
 '도가니탕',
 '된장국',
 '무국',
 '묵국',
 '미역국',
 '배추국',
 '북어국',
 '사골국',
 '삼계탕',
 '새우탕',
 '설렁탕',
 '순대국',
 '어묵국',
 '오징어국',
 '육개장',
 '장어탕',
 '재첩국',
 '조개국',
 '콩나물국',
 '토란국',
 '해장국',
 '갈치찌개',
 '감자탕',
 '김치전골',
 '김치찌개',
 '꽃게탕',
 '낙지전골',
 '동태국',
 '동

In [149]:
food_tmp = []
for name in food_lst:
    hanspell_sent = spell_checker.check(name)
    food_tmp.append(hanspell_sent.checked)

food_tmp

['케이크',
 '치즈빵',
 '카스텔라',
 '추로스',
 '치아바타',
 '와플',
 '찰떡 빵',
 '김밥',
 '덮밥',
 '보리밥',
 '볶음밥',
 '비빔밥',
 '삼각김밥',
 '수수밥',
 '알밥',
 '영양 돌솥밥',
 '오므라이스',
 '자장밥',
 '잡곡밥',
 '잡채밥',
 '주먹밥',
 '짬뽕밥',
 '차조밥',
 '찰밥',
 '초밥',
 '카레라이스',
 '콩나물밥',
 '콩밥',
 '하이라이스',
 '현미밥',
 '회덮밥',
 '가래떡',
 '경단',
 '꿀떡',
 '모둠 찰떡',
 '백설기',
 '송편',
 '시루떡',
 '약식',
 '인절미',
 '절편',
 '증편',
 '찹쌀떡',
 '도넛',
 '마늘빵',
 '머핀',
 '모닝빵',
 '모카빵',
 '베이글',
 '샌드위치',
 '곰보빵',
 '식빵',
 '초코 소라고 빵',
 '크루아상',
 '파이',
 '페이스트리',
 '피자',
 '피자빵',
 '햄버거',
 '호떡',
 '간자장',
 '국수',
 '냉면',
 '떡국',
 '떡만둣국',
 '라면',
 '만두',
 '만둣국',
 '메밀국수',
 '수제비',
 '스파게티',
 '가락국수',
 '가락국수 볶음',
 '월남쌈',
 '자장면',
 '짬뽕',
 '쫄면',
 '칼국수',
 '콩국수',
 '닭죽',
 '수프',
 '오리 죽',
 '전복죽',
 '채소 죽',
 '팥죽',
 '호박죽',
 '흑임자죽',
 '갈비탕',
 '감잣국',
 '곰탕',
 '굴국',
 '김칫국',
 '내장탕',
 '냉국',
 '달 결국',
 '달걀탕',
 '닭곰탕',
 '도가니탕',
 '된장국',
 '뭇국',
 '묵국',
 '미역국',
 '배춧국',
 '북엇국',
 '사골국',
 '삼계탕',
 '새우탕',
 '설렁탕',
 '순댓국',
 '어묵국',
 '오징어 국',
 '육개장',
 '장어탕',
 '재첩 국',
 '조갯국',
 '콩나물국',
 '토란국',
 '해장국',
 '갈치 찌개',
 '감자탕',
 '김치전골',
 '김치찌개',
 '꽃게탕',
 '낙지

In [150]:
food_short_name = []

for name in food_lst:
    for length in [2, 3, 4]:
        cut = name[:length]
        food_short_name.append(cut)

food_short_name = [name for name in food_short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
food_short_name = list(set(food_short_name))
print(len(food_short_name), food_short_name)

1437 ['미숫', '간재미회', '도넛', '어묵국', '치커리', '콩밥', '오이', '북어국', '산자', '쑥갓나물', '배추', '오징어찌', '열무물', '오리탕', '소내장구', '씀바', '보쌈김', '깐쇼새', '골뱅', '호박오가', '초코', '오징어채', '미역초', '두부부침', '탄산음', '호밀', '고춧', '액상커피', '소고', '풋고', '들깨국', '육탕', '치아바', '와플', '돌나', '돈가스', '돌솥비빔', '가래떡', '양장', '미역줄', '방풍나물', '전어구이', '오겹살구', '도가니탕', '어묵', '넙치', '미나', '오리백숙', '기피', '굴국', '순대', '미트', '땅콩조', '곱창구', '달걀탕', '풋고추찜', '깻잎나', '하이라', '페이', '떡볶', '닭튀김', '배추나물', '게국지', '건포', '훈제', '콩비지찌', '밴댕', '또띠', '파래전', '북어채', '유부', '건포도', '카레소', '곤약', '중식잡채', '덮밥', '삼치조', '들깻잎장', '막국', '파전', '상추겉', '버섯', '고사', '게장', '백합', '시루떡', '모싯잎송', '머위', '깻잎나물', '팟타', '미꾸', '치즈스틱', '청포묵무', '양미', '장어', '오이생채', '메밀소바', '에이드', '내장전', '두부부', '보쌈김치', '새우볶음', '칼국수', '전어찜', '상추', '오리죽', '차돌', '오이장', '자몽차', '개고기무', '곱창볶음', '핫도', '느타', '굴무', '충무김밥', '츄러', '페이스', '꼬막', '생선까스', '율무밥', '쥐포튀', '새송', '황태해', '민들레잎', '도가니', '소세지빵', '분짜', '곤약무침', '고추장볶', '돼지껍데', '닭모', '곰장', '도토리묵', '닭날개', '시루', '순대볶음', '냉국', '닭도', '메밀', '애호박전', '소갈비찜', '버섯조림', '우거지', '경단', '사태찜'

In [151]:
drop_name = ['무지','하이','에이','유산','다시','아이','고사','모닝','수제','페이','과ㆍ','삼각','김구','시리','웨이퍼','리소','타르','설렁','도가','개고','월남','치아','밤','매운','세발','기피','스파','닭모','마카','닭날','기타','코코','과ㆍ채','장조','토스','닭도','요구','지리','미소','무장','모자','가자','과ㆍ채주','꼬리','스테이','페이스','마테','소시','시루','샌드','모듬','카스','가죽','소고','총각','탄탄','기스','가자','영양','허브','옥수','부대','동파','단호','약식','다시','무초','비빔','초당','파이','스무']
food_short_name = [col for col in food_short_name if col not in drop_name]
print(len(food_short_name), food_short_name)

1368 ['미숫', '간재미회', '도넛', '어묵국', '치커리', '콩밥', '오이', '북어국', '산자', '쑥갓나물', '배추', '오징어찌', '열무물', '오리탕', '소내장구', '씀바', '보쌈김', '깐쇼새', '골뱅', '호박오가', '초코', '오징어채', '미역초', '두부부침', '탄산음', '호밀', '고춧', '액상커피', '풋고', '들깨국', '육탕', '치아바', '와플', '돌나', '돈가스', '돌솥비빔', '가래떡', '양장', '미역줄', '방풍나물', '전어구이', '오겹살구', '도가니탕', '어묵', '넙치', '미나', '오리백숙', '굴국', '순대', '미트', '땅콩조', '곱창구', '달걀탕', '풋고추찜', '깻잎나', '하이라', '떡볶', '닭튀김', '배추나물', '게국지', '건포', '훈제', '콩비지찌', '밴댕', '또띠', '파래전', '북어채', '유부', '건포도', '카레소', '곤약', '중식잡채', '덮밥', '삼치조', '들깻잎장', '막국', '파전', '상추겉', '버섯', '게장', '백합', '시루떡', '모싯잎송', '머위', '깻잎나물', '팟타', '미꾸', '치즈스틱', '청포묵무', '양미', '장어', '오이생채', '메밀소바', '에이드', '내장전', '두부부', '보쌈김치', '새우볶음', '칼국수', '전어찜', '상추', '오리죽', '차돌', '오이장', '자몽차', '개고기무', '곱창볶음', '핫도', '느타', '굴무', '충무김밥', '츄러', '꼬막', '생선까스', '율무밥', '쥐포튀', '새송', '황태해', '민들레잎', '도가니', '소세지빵', '분짜', '곤약무침', '고추장볶', '돼지껍데', '곰장', '도토리묵', '닭날개', '순대볶음', '냉국', '메밀', '애호박전', '소갈비찜', '버섯조림', '우거지', '경단', '사태찜', '유채물', '멸치', '해물', '김말이', '어육', '무짠지', '비빔막국', 

In [152]:
food = food_lst + food_tmp + food_short_name

In [153]:
food_lst = list(set(food)) # 중복 제거
food_lst

['미숫',
 '간재미회',
 '도넛',
 '어묵국',
 '닭 날개 구이',
 '치커리',
 '콩밥',
 '고구마 전',
 '오이',
 '북어국',
 '산자',
 '생선가스',
 '쑥갓나물',
 '배추',
 '마늘종 볶음',
 '오징어찌',
 '열무물',
 '오리탕',
 '소내장구',
 '노각 무침',
 '씀바',
 '보쌈김',
 '깐쇼새',
 '골뱅',
 '영양 돌솥밥',
 '호박오가',
 '초코',
 '오징어채',
 '미역초',
 '두부부침',
 '탄산음',
 '호밀',
 '고춧',
 '액상커피',
 '새송이버섯볶음',
 '풋고',
 '들깨국',
 '육탕',
 '치아바',
 '와플',
 '돈가스',
 '돌나',
 '돌솥비빔',
 '가래떡',
 '양장',
 '미역줄',
 '방풍나물',
 '전어구이',
 '오겹살구',
 '도가니탕',
 '어묵',
 '넙치',
 '소고기 산적',
 '미나',
 '오리백숙',
 '굴국',
 '순대',
 '모자반무침',
 '미트',
 '가오리콩나물찜',
 '땅콩조',
 '곱창구',
 '달걀탕',
 '풋고추찜',
 '깻잎나',
 '하이라',
 '떡볶',
 '닭튀김',
 '배추나물',
 '게국지',
 '건포',
 '훈제',
 '과일샐러드',
 '콩비지찌',
 '밴댕',
 '또띠',
 '파래전',
 '북어채',
 '유부',
 '도라지생채',
 '토란대나물무침',
 '건포도',
 '카레소',
 '곤약',
 '애호박볶음',
 '중식잡채',
 '덮밥',
 '삼치조',
 '들깻잎장',
 '삼겹살볶음',
 '막국',
 '파전',
 '상추겉',
 '버섯',
 '씀바귀나물무침',
 '게장',
 '백합',
 '시루떡',
 '간재미회무침',
 '모싯잎송',
 '머위',
 '깻잎나물',
 '팟타',
 '치즈스틱',
 '미꾸',
 '청포묵무',
 '양미',
 '병어 찜',
 '장어',
 '오이생채',
 '찰떡 빵',
 '메밀소바',
 '에이드',
 '내장전',
 '두부부',
 '보쌈김치',
 '새우볶음',
 '칼국수',
 '전어찜',
 '고추 조림',
 '

In [154]:
food_lst.append('펩시콜라')

In [155]:
all_food = [word for word in food_lst if len(word) > 1]

In [158]:
if '번' in all_food:
    print("True")
else:
    print("False")

False


In [159]:
all_food.remove('파이')

In [ ]:
train['food'] = train['corrected_reply'].apply(lambda x: next((food for food in all_food if food in x), " "))
train['food']

0          마라
1        부대찌개
2            
3            
4            
         ... 
20719      미역
20720      라면
20721        
20722        
20723        
Name: food, Length: 20724, dtype: object

In [ ]:
train

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food,review,sentiment,score,kiwi_reviews,keybert_reviews
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",마라 하오,죽전점,마라,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,1,0.761931,"[배달, 감사, 맛있]","[(배달, 0.8359), (감사, 0.8353), (맛있, 0.4462)]"
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,"놀부,부대찌,대찌",,부대찌개,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,1,0.586811,"[세트, 많, 좋, 맛있]","[(세트, 0.806), (맛있, 0.5224)]"
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,,,,너무 맛있어요! 자주 시켜 먹게 되네요,1,0.779962,[맛있],"[(맛있, 1.0)]"
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,,,,항상 맛있어요 서비스도 최고,1,0.801292,"[맛있, 서비스, 최고]","[(서비스, 0.9012), (최고, 0.5062), (맛있, 0.5062)]"
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,,,,모든 반찬이 다 맛있습니다!,1,0.762959,"[반찬, 맛있]","[(반찬, 0.9452), (맛있, 0.4905)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20719,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4,맛있게 잘 먹었습니다.,더 노력하는 보둘 미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 ...,,,미역,맛있게 잘 먹었습니다.,1,0.519741,[맛있],"[(맛있, 1.0)]"
20720,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부으시면 국물이 흘러내리기 때문에 혹시라도 모자라면 댁에서 생...,,,라면,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,0,0.542860,"[국물, 많, 국물, 같, 시원하]","[(시원하, 0.7763), (국물, 0.7322)]"
20721,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,,,,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,1,0.766628,"[맛있, 포장, 무엇, 반찬, 짜, 맛있, 구성, 가지, 만족]","[(구성, 0.5366), (만족, 0.4777), (반찬, 0.4172), (가지..."
20722,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,,,,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,1,0.748113,"[많, 덕분, 맛있]","[(덕분, 0.9048), (맛있, 0.4602)]"


# 긍/부정 컬럼화

In [40]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00


In [41]:
from transformers import pipeline

# 모델 불러오기
model_name = "MLOpsEngineer/review-rate-prediction"
classifier = pipeline("sentiment-analysis", model=model_name)

In [ ]:
train[train['별점'] == 4].head(2)

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,부대찌,,부대찌개
19,"야채,단무지 넣지 말라고 요청 드렸는데 주먹밥에 들어거는 김까지 안보내셨네요 담엔 ...",먼저 죄송 합니다 바로 전화 주셨으면 김가루 가져다드릴건데 죄송 합니다 재주문시 꼭...,4,"야채, 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔...",먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,,,야채


In [ ]:
classifier(train['corrected_review'][216])

[{'label': 'LABEL_1', 'score': 0.7982481718063354}]

In [ ]:
classifier(train['corrected_review'][1])

[{'label': 'LABEL_3', 'score': 0.5868110060691833}]

In [46]:
import pandas as pd
from transformers import pipeline

def analyze_reviews(reviews):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in tqdm(reviews):
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = 0 # '부정'
        else:
            sentiment = 1 # '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    tmp = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return tmp

In [ ]:
sentiment_train = analyze_reviews(train['corrected_review'])
sentiment_train

  0%|          | 0/20724 [00:00<?, ?it/s]

,review,sentiment,score
0,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,1,0.761931
1,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,1,0.586811
2,너무 맛있어요! 자주 시켜 먹게 되네요,1,0.779962
3,항상 맛있어요 서비스도 최고,1,0.801292
4,모든 반찬이 다 맛있습니다!,1,0.762959
...,...,...,...
20719,맛있게 잘 먹었습니다.,1,0.519741
20720,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,0,0.542860
20721,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,1,0.766628
20722,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,1,0.748113


In [ ]:
train = pd.concat([train, sentiment_train], axis=1)
train.head()

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food,review,sentiment,score
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",,죽전점,마라,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,1,0.761931
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,"놀부,부대찌,대찌",,부대찌개,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,1,0.586811
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,,,,너무 맛있어요! 자주 시켜 먹게 되네요,1,0.779962
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,,,,항상 맛있어요 서비스도 최고,1,0.801292
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,,,파이,모든 반찬이 다 맛있습니다!,1,0.762959


# keybert로 키워드 추출

## Kiwi 형태소 분석 후 키워드 추출

In [42]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=6f99abe482a84f2316eaff1c01af2ca575612cb1feb9d4392e71b462592634e8
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [43]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [ ]:
tokenized_reviews = []
kiwi_reviews = []

for review in train['corrected_review']:
    results = []
    result = kiwi.analyze(review)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('VA'):
            results.append(token)
    kiwi_reviews.append(results)

In [ ]:
train['kiwi_reviews'] = kiwi_reviews

## keybert

In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=aacea022c171d4c47036cf49b82b859b336adf881e64145bbf55d4125edbb16d
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a50983f865bfa537299ce8184c968f6379743e6b318f4ff850c27234a8bce38c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT()

In [ ]:
train.head()

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food,review,sentiment,score,kiwi_reviews
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",마라 하오,죽전점,마라,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,1,0.761931,"[배달, 감사, 맛있]"
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,"놀부,부대찌,대찌",,부대찌개,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,1,0.586811,"[세트, 많, 좋, 맛있]"
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,,,,너무 맛있어요! 자주 시켜 먹게 되네요,1,0.779962,[맛있]
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,,,,항상 맛있어요 서비스도 최고,1,0.801292,"[맛있, 서비스, 최고]"
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,,,파이,모든 반찬이 다 맛있습니다!,1,0.762959,"[반찬, 맛있]"


In [ ]:
keybert_reviews = []
for keyword in tqdm(train['kiwi_reviews']):
    tmp = []
    keyword = ' '.join(keyword)
    result = kw_model.extract_keywords(keyword,
                              keyphrase_ngram_range=(1, 1),
                              use_mmr=True,
                              diversity=0.7,
                              stop_words=None,
                              top_n=5)
    tmp.append(result)
    keybert_reviews.extend(tmp)

  0%|          | 0/20724 [00:00<?, ?it/s]

In [ ]:
train['keybert_reviews'] = keybert_reviews

In [ ]:
train.head()

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food,review,sentiment,score,kiwi_reviews,keybert_reviews
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",마라 하오,죽전점,마라,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,1,0.761931,"[배달, 감사, 맛있]","[(배달, 0.8359), (감사, 0.8353), (맛있, 0.4462)]"
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,"놀부,부대찌,대찌",,부대찌개,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,1,0.586811,"[세트, 많, 좋, 맛있]","[(세트, 0.806), (맛있, 0.5224)]"
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,,,,너무 맛있어요! 자주 시켜 먹게 되네요,1,0.779962,[맛있],"[(맛있, 1.0)]"
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,,,,항상 맛있어요 서비스도 최고,1,0.801292,"[맛있, 서비스, 최고]","[(서비스, 0.9012), (최고, 0.5062), (맛있, 0.5062)]"
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,,,파이,모든 반찬이 다 맛있습니다!,1,0.762959,"[반찬, 맛있]","[(반찬, 0.9452), (맛있, 0.4905)]"


In [ ]:
train.to_csv(f"{DATA_PATH}0906.csv",index=False) # 저장

- 키워드 빈도수 확인

In [ ]:
keyword = []

for items in train['keybert_reviews']:
    tmp = [item[0] for item in items]
    keyword.extend(tmp)

In [ ]:
from collections import Counter

# all_keywords 리스트에 중복된 키워드의 개수를 세기
keyword_counter = Counter(keyword)

# 빈도수가 높은 순으로 키워드 출력
for keyword, count in keyword_counter.most_common():
    print(f"{keyword}: {count}")

맛있: 12824
배달: 2303
주문: 1432
감사: 1232
빠르: 1203
다음: 776
맛나: 696
여기: 643
서비스: 623
최고: 560
리뷰: 530
고기: 447
오늘: 389
처음: 373
사진: 371
만족: 317
피자: 315
음식: 293
오랜만: 281
그렇: 276
번창: 274
괜찮: 274
사장: 265
생각: 262
국물: 256
메뉴: 237
추천: 235
요청: 230
시간: 229
추가: 229
김치: 212
소스: 211
포장: 210
부드럽: 210
마라탕: 208
가격: 203
치킨: 202
초밥: 198
떡볶이: 190
아쉽: 189
배부르: 186
반찬: 184
친절: 176
치즈: 173
가성비: 173
이벤트: 169
정도: 161
돈가스: 156
맛집: 156
하나: 155
푸짐하: 150
느낌: 144
아이: 139
이번: 135
커피: 132
매콤하: 130
깔끔: 129
곱창: 129
구성: 126
토핑: 124
족발: 123
배송: 121
김밥: 117
사항: 114
냉면: 108
계란: 103
단골: 103
보쌈: 101
닭발: 99
양도: 96
비빔밥: 95
냄새: 93
세트: 91
양념: 89
파스타: 87
매장: 86
주먹밥: 83
꿔바로우: 83
입맛: 83
덮밥: 82
스파게티: 82
가게: 81
샐러드: 80
탕수육: 80
계란찜: 76
다르: 76
저번: 75
새우: 75
재료: 74
이용: 73
필요: 72
육회: 71
다양: 71
저녁: 70
덕분: 69
짬뽕: 69
연어: 68
기분: 68
식사: 67
전화: 66
야채: 64
튀김: 64
저렴하: 64
진하: 64
와플: 64
크림: 62
신경: 62
이상: 61
선택: 61
이렇: 61
국수: 60
김치찌개: 59
단계: 59
비싸: 59
기대: 58
도착: 58
때문: 57
배고프: 57
삼겹살: 57
도우: 56
정성: 55
가지: 55
건강: 54
순대: 54
혼자: 54
전체: 53
막국수: 53
수고: 53
부족:

#-------문장 통째로 학습시켜보기(train_over_0905.csv)----------

In [114]:
train

,고객리뷰,사장댓글,별점
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5
...,...,...,...
21187,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,맛있는 식사가 되셨기를 바랍니다 맛있는 식사 이용해 주셔서 감사합니다,4
21188,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행 업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락주시면 ...,4
21189,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4
21190,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해본 결과 직...,4


In [115]:
train['고객리뷰'] = train['고객리뷰'].astype(str)
train['사장댓글'] = train['사장댓글'].astype(str)

In [55]:
sentiment_train = analyze_reviews(train['고객리뷰'])
sentiment_train

  0%|          | 0/21192 [00:00<?, ?it/s]

,review,sentiment,score
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,1,0.713659
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,1,0.567922
2,너무맛있어요! 자주시켜먹게되네요,1,0.790605
3,항상 맛있어요 서비스도쵝오,1,0.810643
4,모든 반찬이 다 맛있습니다!,1,0.762959
...,...,...,...
21187,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,1,0.808946
21188,맛있게 잘 먹었습니다 맛있게,1,0.611713
21189,맛있게 잘 먹었어요 맛있게 잘,1,0.550359
21190,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,1,0.493313


In [56]:
for i in range(0, 2):
    num_rating = sentiment_train[sentiment_train['sentiment'] == i].shape[0]
    print(f"긍/부정 {i}점 개수: {num_rating}, {num_rating / sentiment_train.shape[0] * 100}%")

긍/부정 0점 개수: 1483, 6.997923744809362%
긍/부정 1점 개수: 19709, 93.00207625519063%


In [116]:
train = pd.concat([train, sentiment_train], axis=1)
train.head()

,고객리뷰,사장댓글,별점,review,sentiment,score
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,1,0.713659
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,1,0.567922
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무맛있어요! 자주시켜먹게되네요,1,0.790605
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도쵝오,1,0.810643
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,1,0.762959


In [117]:
train['사장댓글'] = train['사장댓글'].str.replace(r'\w+\s* 님', '고객님', regex=True)

In [137]:
train['store'] = ""
for name in store_list:
    train[name] = train['사장댓글'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
    train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
train = train.drop(columns=store_list) # 각 가게명 컬럼 삭제

train.head()

,고객리뷰,사장댓글,별점,review,sentiment,score,store
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,1,0.713659,"마라하,마라하오"
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,1,0.567922,"대찌,놀부,놀부부대찌개,부대찌"
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무맛있어요! 자주시켜먹게되네요,1,0.790605,
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도쵝오,1,0.810643,
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,1,0.762959,


In [138]:
train

,고객리뷰,사장댓글,별점,review,sentiment,score,store
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,1,0.713659,"마라하,마라하오"
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,1,0.567922,"대찌,놀부,놀부부대찌개,부대찌"
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무맛있어요! 자주시켜먹게되네요,1,0.790605,
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도쵝오,1,0.810643,
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,1,0.762959,
...,...,...,...,...,...,...,...
21187,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,맛있는 식사가 되셨기를 바랍니다 맛있는 식사 이용해 주셔서 감사합니다,4,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,1,0.808946,
21188,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행 업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락주시면 ...,4,맛있게 잘 먹었습니다 맛있게,1,0.611713,
21189,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4,맛있게 잘 먹었어요 맛있게 잘,1,0.550359,
21190,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해본 결과 직...,4,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,1,0.493313,


In [145]:
train['branch'] = train['사장댓글'].apply(lambda x: next((branch for branch in branch_list if branch in x), " "))
train['branch']

0        죽전점
1           
2           
3           
4           
        ... 
21187       
21188       
21189       
21190       
21191       
Name: branch, Length: 21192, dtype: object

In [160]:
train['food'] = train['사장댓글'].apply(lambda x: next((food for food in all_food if food in x), " "))
train['food']

0         마라
1        부대찌
2           
3           
4           
        ... 
21187       
21188       
21189       
21190       
21191       
Name: food, Length: 21192, dtype: object

In [161]:
train

,고객리뷰,사장댓글,별점,review,sentiment,score,store,branch,food
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,1,0.713659,"마라하,마라하오",죽전점,마라
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,1,0.567922,"대찌,놀부,놀부부대찌개,부대찌",,부대찌
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무맛있어요! 자주시켜먹게되네요,1,0.790605,,,
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도쵝오,1,0.810643,,,
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,1,0.762959,,,
...,...,...,...,...,...,...,...,...,...
21187,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,맛있는 식사가 되셨기를 바랍니다 맛있는 식사 이용해 주셔서 감사합니다,4,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,1,0.808946,,,
21188,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행 업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락주시면 ...,4,맛있게 잘 먹었습니다 맛있게,1,0.611713,,,
21189,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4,맛있게 잘 먹었어요 맛있게 잘,1,0.550359,,,
21190,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해본 결과 직...,4,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,1,0.493313,,,


In [163]:
ng_df = train.loc[train.sentiment == 0] # 부정리뷰
ng_df = ng_df.reset_index(drop=True)
ng_df = ng_df[['고객리뷰','사장댓글','sentiment','store','branch','food']]
ng_df

,고객리뷰,사장댓글,sentiment,store,branch,food
0,"야채,단무지 넣지 말라고 요청 드렸는데 주먹밥에 들어거는 김까지 안보내셨네요 담엔 ...",먼저 죄송 합니다 바로 전화 주셨으면 김가루 가져다드릴건데 죄송 합니다 재주문시 꼭...,0,,,야채
1,폴드포크 제가 제일 좋아하는 메뉴입니다~! 근데 울트라 치즈는 어디에.? 보이지도 ...,고객님 안녕하세요~ 우선 울트라치즈에 치즈맛이 안느껴지셨다니 .치즈가 세번들어가는 ...,0,,,무나
2,아파서 시켰어요 저는 참깨김후레이크? 그거가 저랑 잘 안맞네요.,안녕하세요! 찾아주셔서 감사합니다 그런데 고객님과 맞지 않는 부분으로 아쉬움이 남으...,0,,,
3,예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십년만에 주문을 해봅니다. 양 넉...,안녕하세요 네 잘 알겠습니다 잘 살펴보겠습니다 감사합니다,0,,,
4,국은 시원하게 맛있었어요 국물양이 더 많았으면 더 좋았겠습니다,맛있게 드셨다니 감사드립니다. 매장과 같이 최대한 드리고있습니다. 배달용기에 국물을...,0,,,
...,...,...,...,...,...,...
1478,예전에는 반찬도 많이 주셨는데 요즘에는 반찬이 없네요,항상 고객분들의 의견에 귀를 기울이고 더 좋은 음식을 준비하기 위해 노력하는 한편 ...,0,,,
1479,부대찌개 맛은 그냥 그런데 속에 든 햄은 꽤 많은데 김치가 좀 모자랐어요 저희 커플...,저희는 항상 10분 안에 모든 메뉴가 준비되어 있지만 배달대행업체의 특성상 동시간대...,0,,,
1480,근데 시간이 너무 오래 걸려요 근데 역씨 놀부는 너무 인기가 많아요,항상 푸짐하고 좋은 음식을 제공해 드리는 곳이 되기 위해 노력하겠습니다.,0,,,
1481,진짜 짜증 나게 하는 게 배달 기사 아저씨가 진짜 최악이에요.,저희 매장을 항상 방문해 주시는 분들께 감사의 마음을 전하며 더 좋은 요리와 따뜻한...,0,,,


In [164]:
ps_df = train.loc[train.sentiment == 1] # 긍정리뷰
ps_df = ps_df.sample(frac=1).reset_index(drop=True) # 데이터 랜덤 섞기
ps_df = ps_df[:3000][['고객리뷰','사장댓글','sentiment','store','branch','food']] # 3000개만 뽑기
ps_df

,고객리뷰,사장댓글,sentiment,store,branch,food
0,저희는 칼국수 여기에서만 주문해서 먹어요. 국물이 너무 맛있구요 사장님이 친절해서인...,찐~감사요~,1,,,
1,돈가스회관 떡볶이에 돈가스와 치킨가스를 곁들여 먹으니 최상의 조합이네요! 완전 맛있...,고객님 항상 떡볶이를 맛나게 드시는 울 고객님 감사드립니다. 떡볶이와 돈까스가 은근...,1,"돈까,돈까스회,돈까스회관",,떡볶
2,2년만에 다시 먹을 수 있으니 너무 벅차올 거 같아요 코로나 젤 심에 시기에 가장 ...,저팔계 중국 마라탕 덮밥 요리 입니다. 저희 음식을 맛있게 드시고 소중한 사진리뷰...,1,"저팔,저팔계",,덮밥
3,빽다방은 늘 옳아요~,"안녕하세요. 대전역점 빽다방입니다. 맛있게 드셨다니 기분이 좋아요 싸고, 크고, 맛...",1,"빽다,빽다방",대전역점,
4,매주 이용하는곳 입니다,감사합니다 리뷰님.,1,,,
...,...,...,...,...,...,...
2995,"맛, 서비스 친절 모두 최고에요. 양도 넉넉하고 사이드 메뉴 구성도 좋습니다.",아직 부족한게많은데 웃으면서 이해해주시니 감사할따름입니다! 다음에도 고객님의 만족...,1,홍스,,
2996,맛있게 먹었어요.,감사합니다 항상 변함없는 맛으로 찾아 뵙겠습니다,1,,,
2997,맛있어요,맛있게 드셔주시고 시간 내서 소중한 리뷰 달아주셔서 너무너무 감사합니다 ~.~ 고객...,1,,,
2998,튀김류가 맛있어요,맛있게 드셔 주셔서 감사합니다. 소중한 리뷰도 감사드리고 앞으로도 많은 관심과 이용...,1,국수나무,,국수


In [165]:
ng_df.to_csv(f"{DATA_PATH}0908_ng_df.csv",index=False) # 저장
ps_df.to_csv(f"{DATA_PATH}0908_ps_df.csv",index=False) # 저장

In [167]:
ng_df.isnull().sum(), ps_df.isnull().sum()

(고객리뷰         0
 사장댓글         0
 sentiment    0
 store        0
 branch       0
 food         0
 dtype: int64,
 고객리뷰         0
 사장댓글         0
 sentiment    0
 store        0
 branch       0
 food         0
 dtype: int64)